In [43]:
import os,sys,string,re
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt

to_space= '''&,()+'"/'''
trans=string.maketrans(to_space, ' '*len(to_space))
ps = PorterStemmer()

os.chdir('/data/arpit.goel/31_PricePrediction/03.Submissions')
pd.set_option('display.float_format', lambda x: '%.1f' % x)

df_raw_train=pd.read_csv('../input/train.tsv',delimiter='\t')
df_raw_test=pd.read_csv('../input/test.tsv',delimiter='\t')
df_raw_train['log_price']=np.clip(np.log(1+df_raw_train['price']),1,5)

ins=df_raw_train.sample(frac=0.8,random_state=200)
oos=df_raw_train.drop(ins.index)
oot=df_raw_test

In [46]:
def get_free_text_vars(df,var,label):
    df['ft_0_%s'%label]=df[var].astype(np.str).str.len()                                                             # Num characters
    df['ft_1_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[%s]'%string.printable,'',x)).str.len()         # Num non printable chars
    df['ft_2_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^#$%&*:;<=>?@\^_`|~]','',x)).str.len()         # Num Special Characters
    df['ft_3_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^0-9]','',x)).str.len()                        # Num numerals
    df['ft_4_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^a-z]','',x)).str.len()                        # Num lower case
    df['ft_5_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^A-Z]','',x)).str.len()                        # Num upper case
    df['ft_6_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^ ]','',x)).str.len()                          # Num spaces
    df['%s_clean'%label]=df[var].astype(np.str).str.lower().map(lambda x: re.sub('[^ 0-9a-z]',' ',x))                 # Clean Text
    
for data in [ins,oos,oot]:
    for var,label in [('name','name'),('item_description','desc')]:
        get_free_text_vars(data,var,label)


In [83]:
all_words=pd.concat([ins['name_clean'],ins['desc_clean']]).values
all_words=ins['desc_clean'].values
all_words=[y for x in all_words for y in x.split()]
all_words=pd.Series(Counter(all_words))

In [58]:
all_words.sort_values(ascending=False).to_csv('../02.Profile/6.AllWords.csv')

In [84]:
print all_words.shape
print all_words[all_words>100].shape


(142966,)
(10084,)


In [86]:
def stem_word(x):
    try:
        return ps.stem(x)
    except:
        return x

stemmed_words=all_words.reset_index()
stemmed_words.columns=['word','count']
stemmed_words['stem']=stemmed_words['word'].map(stem_word)
stemmed_words=stemmed_words.groupby('stem')['count'].sum()

In [87]:
print stemmed_words.shape
print stemmed_words[stemmed_words>100].shape


(117178,)
(7764,)


In [77]:
import nltk.corpus

nltk.corpus.sentiwordnet


<SentiWordNetCorpusReader in u'.../corpora/sentiwordnet' (not loaded yet)>